In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-01 23:43:53.920350: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-01 23:43:55.385142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-01 23:43:55.385255: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-01 23:43:55.385262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT li

In [7]:
data_folder = "./results/split_slices/"
results_folder = "./results/PASTE2/"
figures_folder = "./results/figures/PASTE2"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(figures_folder).mkdir(parents=True, exist_ok=True)

In [2]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|████████████████████████| 59/59 [03:29<00:00,  3.55s/it]


In [3]:
spatial_key = "spatial"
anno_key = "class"
palette = slices[0].uns[f"{anno_key}_palette"]

In [4]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [5]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [ ]:
## Perform the Moscot alignment
sampling_num = 5000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
        fixed_s=0.99,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1), np.sum(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                 | 0/58 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.893403e+00|0.000000e+00|0.000000e+00
    1|1.207100e+00|2.225420e+00|2.686304e+00
    2|1.105224e+00|9.217671e-02|1.018759e-01
    3|1.004427e+00|1.003525e-01|1.007967e-01
    4|1.001866e+00|2.555898e-03|2.560668e-03
    5|1.001856e+00|1.089936e-05|1.091959e-05
    6|1.001855e+00|8.993193e-08|9.009879e-08
 1000|1.001855e+00|0.000000e+00|0.000000e+00


  2%|▍                     | 1/58 [02:24<2:17:02, 144.25s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.768861e+00|0.000000e+00|0.000000e+00
    1|1.267772e+00|1.972823e+00|2.501090e+00
    2|1.152712e+00|9.981656e-02|1.150598e-01
    3|1.098687e+00|4.917198e-02|5.402464e-02
    4|1.097706e+00|8.937080e-04|9.810290e-04
    5|1.097697e+00|8.903430e-06|9.773265e-06
 1000|1.097697e+00|0.000000e+00|0.000000e+00


  3%|▊                     | 2/58 [04:27<2:03:19, 132.13s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


...

PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.220285e+00|0.000000e+00|0.000000e+00
    1|1.056270e+00|2.048732e+00|2.164014e+00
    2|9.885132e-01|6.854435e-02|6.775699e-02
    3|9.013461e-01|9.670768e-02|8.716709e-02
    4|8.982538e-01|3.442526e-03|3.092262e-03
    5|8.982309e-01|2.551035e-05|2.291418e-05
    6|8.982306e-01|3.009435e-07|2.703167e-07
 1000|8.982306e-01|0.000000e+00|0.000000e+00


 90%|██████████████████▊  | 52/58 [2:19:11<16:27, 164.60s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.385952e+00|0.000000e+00|0.000000e+00
    1|1.206708e+00|1.805941e+00|2.179244e+00
    2|1.146865e+00|5.217958e-02|5.984296e-02
    3|1.069371e+00|7.246777e-02|7.749490e-02
    4|1.063570e+00|5.454050e-03|5.800763e-03
    5|1.063414e+00|1.461907e-04|1.554612e-04
    6|1.063402e+00|1.157749e-05|1.231152e-05
    7|1.063401e+00|6.718937e-07|7.144927e-07
    8|1.063400e+00|9.556769e-07|1.016267e-06
    9|1.063394e+00|5.603664e-06|5.958904e-06
   10|1.063394e+00|6.527966e-07|6.941798e-07
   11|1.063394e+00|3.230418e-08|3.435206e-08
 1000|1.063394e+00|0.000000e+00|0.000000e+00


 91%|███████████████████▏ | 53/58 [2:23:12<15:36, 187.38s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.646854e+00|0.000000e+00|0.000000e+00
